<a href="https://colab.research.google.com/github/remotesensinginfo/pb_gee_tools/blob/main/examples/applications/mangrove_class/07_Mangrove_Apply_Asset_Class_to_Tiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The Google Earth Engine module
import ee

# The datetime module is used to specify the dates
# to search for imagery
import datetime

# Import the geemap (https://geemap.org/) module which
# has a visualisation tool
import geemap

# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas

# The colab module to access data from your google drive
from google.colab import drive

In [ ]:
try:
  import pb_gee_tools
  import pb_gee_tools.datasets
  import pb_gee_tools.convert_types
except:
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip install ./pb_gee_tools/.
  import pb_gee_tools
  import pb_gee_tools.datasets
  import pb_gee_tools.convert_types

In [ ]:
ee_prj_name = "ee-pb-dev"  # <==== Replace this with your own EE project string
ee.Authenticate()
ee.Initialize(project=ee_prj_name)

In [ ]:
drive.mount("/content/drive")

In [ ]:
# The file path on google drive for ROI output vector file.
vec_tiles_file = "/content/drive/MyDrive/mangrove_chng_cls/cls_tiles.geojson"

# Start and End date
start_date = datetime.datetime(year=2020, month=1, day=1)
end_date = datetime.datetime(year=2020, month=12, day=31)

# Output no data value
no_data_val = 0.0

# Define the output directory on gdrive
out_gdrive_dir = "mangrove_chng_cls_tiles"

In [ ]:
# Load the classifier from asset
cls_mdl_asset_id = f'projects/{ee_prj_name}/assets/mng_test_rf_cls'
rf_cls_mdl = ee.Classifier.load(cls_mdl_asset_id)

In [ ]:
# Read the vector layer and make sure it is project using WGS84 (EPSG:4326)
vec_cls_roi_gdf = geopandas.read_file(vec_tiles_file).to_crs(4326)

vec_cls_roi_gdf.head()

In [ ]:
# Get the list of tile names from the input vector
tile_names = vec_cls_roi_gdf["tile_names"]
# Loop through the tiles.
for tile_name in tile_names:
  print(tile_name)
  # Subset the input vector to just the tile geometry
  vec_tile_gdf = vec_cls_roi_gdf[vec_cls_roi_gdf["tile_names"] == tile_name]

  # Get layer bbox: minx, miny, maxx, maxy
  gp_roi_bbox = vec_tile_gdf.total_bounds

  # Create the GEE geometry from the bbox.
  roi_cls_west = gp_roi_bbox[0]
  roi_cls_east = gp_roi_bbox[2]
  roi_cls_north = gp_roi_bbox[3]
  roi_cls_south = gp_roi_bbox[1]
  cls_tile_aoi = ee.Geometry.BBox(roi_cls_west, roi_cls_south, roi_cls_east, roi_cls_north)

  # Load the Sentinel-2 imagery.
  s2_img_col = pb_gee_tools.datasets.get_sen2_sr_collection(
      aoi=cls_tile_aoi,
      start_date=start_date,
      end_date=end_date,
      cloud_thres = 70,
  )

  # Create a median composite clipped to the ROI.
  s2_med_image = s2_img_col.median().clip(cls_tile_aoi).unmask(no_data_val)

  # Get the list of image bands.
  img_bands = s2_med_image.bandNames()

  # Apply the Classification.
  cls_rslts_img = s2_med_image.classify(rf_cls_mdl)

  # Define the output file name
  out_cls_img_file = f"{tile_name}_asset_mdl_cls"

  # Export the classification image to your Google Drive
  task = ee.batch.Export.image.toDrive(
          image=cls_rslts_img,
          description=out_cls_img_file,
          folder=out_gdrive_dir,
          scale=10,
          region=cls_tile_aoi,
          fileFormat="GeoTIFF",
          formatOptions={"cloudOptimized": True, 'noData': 0},
  )
  task.start()
